Train set: use the same from the previous exercise
Test set: Parse XML/RSS New York Times feed and take {doc_id: title, doc: description}
Extract keyphrases for the test set
Generate html page from the keyphrases extracted

Plotly export html:
* https://stackoverflow.com/questions/36262748/python-save-plotly-plot-to-local-file-and-insert-into-html
* https://plot.ly/python/v3/html-reports/

In [1]:
import re
import operator
import json
import itertools
import spacy
import requests
import networkx as nx
import numpy as np
import pandas as pd
from glob import glob
from nltk import ngrams, pos_tag, word_tokenize, everygrams
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from xml.etree import ElementTree
from collections import Counter
from math import log

from tqdm import tqdm_notebook as tqdm

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 500)

In [2]:
from nltk.stem.snowball import SnowballStemmer
sno = SnowballStemmer('english')
nlp = spacy.load('en_core_web_sm')
lemmatizer = WordNetLemmatizer()

In [3]:
def read(directory):
    docs = {}
    for doc_path in tqdm(glob(f'{directory}/*.xml')):
        doc = ElementTree.parse(doc_path)
        sentences = []
        for sentence in doc.find('document').find('sentences').findall('sentence'):
            sentences.append(' '.join([token.find('lemma').text.lower()
                                       for token in sentence.find('tokens').findall('token')]))

        docs[doc_path.split('/')[-1].split('.')[0]] = '\n'.join(sentences)
    return docs


train_docs = read('../Project1/ake-datasets/datasets/Inspec/train')
# test_docs = read('../Project1/ake-datasets/datasets/Inspec/test')
len(train_docs)

1000

In [4]:
CATEGORY = 'Dance'
URL = f'https://rss.nytimes.com/services/xml/rss/nyt/{CATEGORY}.xml'
r = requests.get(URL)
tree = ElementTree.fromstring(r.content)

In [5]:
items = []
test_docs = {}
for item in tree.find('channel').findall('item'):
    try:
        image = item.find('{http://search.yahoo.com/mrss/}content').attrib['url']
    except:
        image = None
    title = item.find('title').text
    description = item.find('description').text
    items.append({
        'title': title,
        'link': item.find('link').text, 
        'description': description,
        'image': image,
    })
    
    doc = nlp(description.lower())
    test_docs[title] = ' '.join([t.lemma_ for t in doc if t.is_alpha and not t.is_stop])

len(test_docs)

19

In [6]:
train_candidates = {doc_id: [' '.join(gram) for gram in everygrams(doc.split(), 1, 3)] for doc_id, doc in train_docs.items()}
train_frequencies = {doc_id: Counter([' '.join(gram) for gram in everygrams(doc.split(), 1, 3)])
                    for doc_id, doc in train_docs.items()}

test_candidates = {doc_id: [' '.join(gram) for gram in everygrams(doc.split(), 1, 3)] for doc_id, doc in test_docs.items()}
test_frequencies = {doc_id: Counter([' '.join(gram) for gram in everygrams(doc.split(), 1, 3)])
                    for doc_id, doc in test_docs.items()}

In [7]:
def tf(d, t, frequencies):
    return 1. * frequencies[d][t] / frequencies[d].most_common(1)[0][1]

def idf(t, frequencies):
    N = 1. * len(frequencies)
    nt = sum(1 for doc in frequencies.values() if t in doc)
    return log(N / nt) if N != 0 and nt != 0 else 0

dls = {}
def bm25(t, d, frequencies, background_frequencies, k1=1.2, b=0.75):
    """
    :param t: term
    :param d: document-id in test dataset
    
    ftd = f(t, d): term frequency
    avgdl = mean([len(doc) for doc in train])
    N = len(train)
    nt = n(t) = sum(1 for doc in train if t in doc)
    """
    N = len(background_frequencies)
    nt = sum(1 for doc in background_frequencies.values() if t in doc)
    # Dangerous but works for our train/test split
    if len(background_frequencies) not in dls:
        dls[len(background_frequencies)] = np.mean([sum(freq.values()) for freq in background_frequencies.values()])
    avgdl = dls[len(background_frequencies)]
    
    ftd = 1. * frequencies[d][t] / frequencies[d].most_common(1)[0][1]
    ld = sum(frequencies[d].values())
    
    tf = (ftd * (k1 + 1)) / (ftd + k1 * (1 - b + b * ld / avgdl))
    idf = log((N - nt + 0.5) / (nt + 0.5))
    return tf * idf

In [8]:
tf(t='datum', d='1390', frequencies=train_frequencies), \
idf(t='datum', frequencies=train_frequencies), \
bm25(t='datum', d='1390', frequencies=train_frequencies, background_frequencies=train_frequencies)

(0.8, 1.9105430052180221, 1.8559239707875061)

In [9]:
test_data = pd.DataFrame([
    {'id': doc_id + ':::' + str(i), 'token': candidate} 
    for doc_id, candidates in test_frequencies.items()
        for i, candidate in enumerate(candidates)
])
test_data.set_index('id', inplace=True)
test_data.head(3)

,token
id,
"Review: The Tap Dancer Dormeshia Finds Her Groove, and Then Some:::0",stellar
"Review: The Tap Dancer Dormeshia Finds Her Groove, and Then Some:::1",swing
"Review: The Tap Dancer Dormeshia Finds Her Groove, and Then Some:::2",joyce


In [10]:
test_data['tf'] = [tf(d=i.split(':::')[0], t=row['token'], frequencies=test_frequencies) for i, row in tqdm(test_data.iterrows(), total=len(test_data))]
test_data['idf'] = [idf(t=row['token'], frequencies=test_frequencies) for i, row in tqdm(test_data.iterrows(), total=len(test_data))]
test_data['tf-idf'] = test_data['tf'] * test_data['idf']
test_data['bm25'] = [bm25(t=row['token'], d=i.split(':::')[0], frequencies=test_frequencies, background_frequencies=train_frequencies) for i, row in tqdm(test_data.iterrows(), total=len(test_data))]
test_data['len'] = [len(row['token']) for i, row in tqdm(test_data.iterrows(), total=len(test_data))]
test_data.head(2)

,token,tf,idf,tf-idf,bm25,len
id,,,,,,
"Review: The Tap Dancer Dormeshia Finds Her Groove, and Then Some:::0",stellar,1.0,2.944439,2.944439,12.202547,7
"Review: The Tap Dancer Dormeshia Finds Her Groove, and Then Some:::1",swing,1.0,2.944439,2.944439,10.437337,5


## Add graph centrality score for each candidate

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
from fasttext import load_model
model = load_model('./cc.en.300.bin')

In [12]:
def get_ranks(doc_id, doc, get_weight, get_personalization=None, weight='weight'):
    G = nx.Graph(name=doc_id)
    doc = nlp(doc)
    
    for sentence in doc.sents:
        tokens = [str(t) for t in sentence if t.is_alpha and not t.is_stop]
        grams = everygrams(tokens, min_len=1, max_len=3)
        grams = [' '.join(g) for g in grams]
        G.add_nodes_from(grams)
        
        edges = list(itertools.combinations(grams, 2))
        weighted_edges = [(v1, v2, get_weight(v1, v2)) for v1, v2 in edges]
        G.add_weighted_edges_from(weighted_edges)
    
    personalization = {node: get_personalization(node) for node in G.nodes} if get_personalization else None
    rank = nx.pagerank(G, alpha=1-0.15, max_iter=50, weight=weight, personalization=personalization)
    return rank

In [13]:
def weight(w1, w2):
    v1 = model.get_sentence_vector(w1)
    v2 = model.get_sentence_vector(w2)
    return cosine_similarity([v1], [v2])[0]

In [14]:
cur_doc_id = -1
centralities = []
doc = None
for i, row in tqdm(test_data.iterrows(), total=len(test_data)):
    doc_id = i.split(':::')[0]
    if cur_doc_id != doc_id:
        cur_doc_id = doc_id
        doc = test_docs[doc_id]
        d = nlp(doc)
        d = ' '.join([str(t) for t in d if t.is_alpha and not t.is_stop])
        doc_ranks = get_ranks(doc_id, doc, get_weight=weight, 
                              get_personalization=lambda x: 1 / (1 + log(1 + d.find(x))))

    token = row['token']
    if token not in doc_ranks: # To remove stop words and non alphanumeric tokens later on
        centralities.append(0)
    else:
        try:
            centralities.append(doc_ranks[token][0])
        except:
            centralities.append(doc_ranks[token])

test_data['centrality'] = centralities
test_data.head()

,token,tf,idf,tf-idf,bm25,len,centrality
id,,,,,,,
"Review: The Tap Dancer Dormeshia Finds Her Groove, and Then Some:::0",stellar,1.0,2.944439,2.944439,12.202547,7,0.024168
"Review: The Tap Dancer Dormeshia Finds Her Groove, and Then Some:::1",swing,1.0,2.944439,2.944439,10.437337,5,0.020537
"Review: The Tap Dancer Dormeshia Finds Her Groove, and Then Some:::2",joyce,1.0,2.944439,2.944439,12.202547,5,0.033309
"Review: The Tap Dancer Dormeshia Finds Her Groove, and Then Some:::3",theater,1.0,1.152680,1.152680,10.437337,7,0.016148
"Review: The Tap Dancer Dormeshia Finds Her Groove, and Then Some:::4",join,1.0,2.944439,2.944439,8.345158,4,0.015984


In [15]:
def avg_precisoin(pred, targ):
    pred = set([sno.stem(p) for p in pred])
    targ = set([sno.stem(t) for t in targ])
    res, nb_correct = 0, 0
    for i, p in enumerate(pred):
        if p in targ:
            nb_correct += 1
        res += nb_correct / (i + 1)
    return 1. / len(targ) * res

In [16]:
len(test_data)

594

In [17]:
test_data['doc_id'] = [i.split(':::')[0] for i in test_data.index]

In [18]:
pd.options.mode.chained_assignment = None  # default='warn'
test_doc_groups = test_data.copy()
test_doc_groups.reset_index(inplace=True)
del test_doc_groups['id']
test_doc_groups = test_doc_groups.groupby('doc_id')


def RRFScore(ranks):
    return sum(1. / (50 + r) for r in ranks)


def rank(arr):
    order = arr.argsort()
    ranks = order.argsort()
    return ranks


def score(doc_id, df):
    ranking = {}
    for i, row in df.iterrows():
        ranking[row['token']] = RRFScore([row['tf'], row['idf'], row['tf-idf'], row['bm25'], 
                                          row['len'], row['centrality']])
        
    top = sorted(ranking.items(), key=operator.itemgetter(1), reverse=True)[:5]
    top_keywords = [k for k, s in top]
#     print(doc_id)
#     print(top_keywords)
#     print(target[doc_id])
    return top_keywords


keywords = {}
for name, group in tqdm(test_doc_groups):    
    group['tf']         = rank(-group['tf'].values)
    group['idf']        = rank(-group['idf'].values)
    group['tf-idf']     = rank(-group['tf-idf'].values)
    group['bm25']       = rank(-group['bm25'].values)
    group['len']        = rank(-group['len'].values)
    group['centrality'] = rank(-group['centrality'].values)
    
    keywords[name] = score(name, group)
    
print(keywords)


{'6 Dance Performances to See in N.Y.C. This Weekend': ['performance happen weekend', 'dance performance happen', 'guide dance performance', 'week ahead', 'dance performance'], '9 Dance Performances to See in N.Y.C. This Weekend': ['performance happen weekend', 'dance performance happen', 'guide dance performance', 'week ahead', 'dance performance'], 'A Choreographer Dances Out His Feelings About Race and Religion': ['kyle marshall rise', 'kyle', 'company brooklyn', 'marshall rise young', 'brooklyn academy'], 'After Misty Comes Marie: Breaking Barriers in ‘The Nutcracker’': ['black marie', 'marie young', 'young heroine', 'year time new', 'ballet nutcracker'], 'Can Dance Make a More Just America? Donald Byrd Is Working on It': ['choreographer commitment dance', 'museum seattle', 'commitment dance catalyst', 'seattle new', 'work alvin'], 'Celebrating Donald McKayle, a Modern-Dance Trailblazer': ['struggle resilience hope', 'theme struggle resilience', 'exploration theme struggle', 'life

## Generate the HTML

In [45]:
data = pd.DataFrame(items)
data['keywords'] = [keywords[row['title']] for i, row in data.iterrows()]
data.head(2)

,title,link,description,image,keywords
0,"Review: The Tap Dancer Dormeshia Finds Her Groove, and Then Some",https://www.nytimes.com/2019/12/04/arts/dance/dormeshia-and-still-you-must-swing-review.html?emc=rss&partner=rss,"In the stellar “And Still You Must Swing” at the Joyce Theater, she is joined by Jason Samuels Smith, Derick K. Grant and Camille A. Brown.",https://static01.nyt.com/images/2019/12/04/arts/04dormeshia-2/04dormeshia-2-moth.jpg,"[samuels smith derick, smith derick grant, jason samuels smith, join jason samuels, swing joyce theater]"
1,Can Dance Make a More Just America? Donald Byrd Is Working on It,https://www.nytimes.com/2019/11/28/arts/dance/donald-byrd-exhibition.html?emc=rss&partner=rss,The choreographer’s commitment to dance as a catalyst for social change can be seen at a museum show in Seattle and in a new work for the Alvin Ailey company.,https://static01.nyt.com/images/2019/12/01/arts/01DONALD-BYRD1/01DONALD-BYRD1-moth.jpg,"[choreographer commitment dance, museum seattle, commitment dance catalyst, seattle new, work alvin]"


In [66]:
html = """
<style>
    body {
        margin-left: auto;
        margin-right: auto;
        text-align: center;
        display: inline-block;
    }
    .card {
        padding: 20px;
        margin: 0 auto;
    }
    .center {
        display: block;
        margin-left: auto;
        margin-right: auto;
        text-align:center;
    }
</style>
<html>
<body>
    <table>
        """

for i, row in data.iterrows():
    if i % 5 == 0:  html += '\n<tr>\n'
    
    title = row['title']
    url = row['link']
    image = row['image']
    html += f'''
    <td class="card">
        <br/><br/>
        <p>{title}</p>
        <img src="{image}" class="center" />
        <a href="{url}" class="center" target="_blank">Read Full article</a>
        <ul style="list-style-type:disc;">
    '''
    
    for key in row['keywords']:
        html += f'<li>{key}</li>'
    html += '''
        </ul>
    </td>
    '''
    
    if i % 5 == 4:  html += '\n</tr>\n'


html += """
    </table>
</body>
</html>
"""
with open('keywords.html', 'w') as f:
    f.write(html)